This notebook generates the dataset for the step selection model.

In [1]:
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
import pickle
import os
import glob
import utm
from pathlib import Path
import math
from math import dist
import matplotlib
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from osgeo import gdal
gdal.UseExceptions()

#from functions.data_prep import calculate_initial_compass_bearing
import stepselector.preprocessing as pp
#from stepselector.preprocessing import extract_observed_steps, simulate_fake_steps, dot, ang, get_observer_and_step_info, calculate_zebra_heights, road_or_no, step_slope, get_social_info

In [2]:
server_mount = '/home/blair/server/herd_hover'

In [3]:
# If obs_to_process = None, all of the raw track files in the data/raw_tracks folder will be processed
obs_to_process = None 

# Alternatively, a subset of observations can be processed
#obs_to_process = ['ob015', 'ob027', 'ob074', 'ob088']

In [4]:
# Define directories
data_folder = os.path.join(server_mount, 'zebra_movement_data')
raw_tracks_directory = os.path.join(data_folder, 'raw_tracks')
rasters_directory = os.path.join(data_folder, 'rasters')
#map_directory = '/home/blair/server/herd_hover/3D_mapping/big_maps'
#rasters_directory = '/home/blair/server/herd_hover/fine_scale_movement/maps'
observed_steps_directory = os.path.join(data_folder, 'five_meter_steps', 'observed')
simulated_steps_directory = os.path.join(data_folder, 'five_meter_steps', 'simulated')

# Define files
ob_metadata_file = os.path.join(data_folder, 'observation_metadata.csv')
track_metadata_file = os.path.join(data_folder, 'track_metadata.csv')

In [5]:
# Step 1 - extract observed steps
pp.extract_observed_steps(step_length = 5, 
                       raw_tracks_directory = raw_tracks_directory, 
                       save_directory = observed_steps_directory, 
                       obs_to_process = obs_to_process)

  0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
# Step 2 - simulate fake steps
pp.simulate_fake_steps(n_steps = 5, 
                    observed_steps_directory = observed_steps_directory, 
                    save_directory = simulated_steps_directory, 
                    rasters_directory = rasters_directory, 
                    ob_metadata_file = ob_metadata_file, 
                    obs_to_process = obs_to_process)

  0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
# Step 3 - get observer and step info
pp.get_observer_and_step_info(observed_steps_directory = observed_steps_directory,
                           simulated_steps_directory = simulated_steps_directory,
                           ob_metadata_file = ob_metadata_file,
                           obs_to_process = obs_to_process)

  0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
# Step 4 - calculate zebra observation heights for visibility analyses
pp.calculate_zebra_heights(observed_steps_directory = observed_steps_directory,
                        simulated_steps_directory = simulated_steps_directory,
                        rasters_directory = rasters_directory,
                        ob_metadata_file = ob_metadata_file,
                        obs_to_process = obs_to_process)

  0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
# Step 5 - get road info
pp.road_or_no(observed_steps_directory = observed_steps_directory,
              simulated_steps_directory = simulated_steps_directory,
              rasters_directory = rasters_directory,
              ob_metadata_file = ob_metadata_file,
              obs_to_process = obs_to_process)

  0%|          | 0/4 [00:00<?, ?it/s]

In [10]:
# Step 6 - get step slope
pp.step_slope(observed_steps_directory = observed_steps_directory,
           simulated_steps_directory = simulated_steps_directory,
           rasters_directory = rasters_directory,
           ob_metadata_file = ob_metadata_file,
           obs_to_process = obs_to_process)

  0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
# Step 6 - get social info
pp.get_social_info(observed_steps_directory = observed_steps_directory,
                simulated_steps_directory = simulated_steps_directory,
                raw_tracks_directory = raw_tracks_directory,
                rasters_directory = rasters_directory,
                ob_metadata_file = ob_metadata_file,
                track_metadata_file = track_metadata_file,
                obs_to_process = obs_to_process)

  0%|          | 0/4 [00:00<?, ?it/s]